In [ ]:
from pathlib import Path
import os
if not str(Path.cwd()).endswith('/Cloud-Telescope'):
    os.chdir(Path.cwd().parent)
    print(f'New working dir is {Path.cwd()}')    
import importlib
import pandas as pd

import src.d01_data.read_files  
importlib.reload(src.d01_data.read_files)
import src.d01_data.save_files
importlib.reload(src.d01_data.save_files)
import src.d01_data.log_names_enum
importlib.reload(src.d01_data.log_names_enum)

import src.d04_analysation.count_percent
importlib.reload(src.d04_analysation.count_percent)

import src.d02_intermediate.filter_telescope_ips
importlib.reload(src.d02_intermediate.filter_telescope_ips)

import src.d04_analysation.scanning
importlib.reload(src.d04_analysation.scanning)

import src.d04_analysation.arrival_rate
importlib.reload(src.d04_analysation.arrival_rate)

import src.d04_analysation.scnas_mapped_to_country
importlib.reload(src.d04_analysation.scnas_mapped_to_country)

import src.d04_analysation.scans_proto_country
importlib.reload(src.d04_analysation.scans_proto_country)

import src.d04_analysation.find_scanner
importlib.reload(src.d04_analysation.find_scanner)

import src.d03_visiualisation.tabel_visiualisation
importlib.reload(src.d03_visiualisation.tabel_visiualisation)

import src.d03_visiualisation.bar_diagram
importlib.reload(src.d03_visiualisation.bar_diagram)

import src.d03_visiualisation.head_map
importlib.reload(src.d03_visiualisation.head_map)

import src.d03_visiualisation.graf_arrival_rate
importlib.reload(src.d03_visiualisation.graf_arrival_rate)

### find Port and Address Scanner 

In [ ]:
import pandas as pd
from src.d01_data.read_files import get_log_columns
from src.d01_data.log_names_enum import *
from src.d01_data.read_files import *
from src.config import *

scan_file: str = f'/home/tobias/work/Cloud-Telescope/data/raw/cloud_merged_pcap/zeek_scanner_folders/zeek_scan_addr-4.0_20min_p-4.0_20min/notice.log'


scan_columns: list[str] = get_log_columns(f'{scan_file}')
scan: pd.DataFrame = pd.read_csv(
    f'{scan_file}',
    delimiter='\t',
    comment='#',
    names=scan_columns,
    low_memory=False
)
zeek_port_scan = scan[scan['note'] == 'ScanUDP::Port_Scan']
zeek_port_scan = zeek_port_scan.drop_duplicates(subset='src', keep='first')
zeek_port_scan = zeek_port_scan.sort_values(by='ts', ascending=True)
zeek_port_scan = zeek_port_scan.reset_index(drop=True)

zeek_address_scan = scan[scan['note'] == 'ScanUDP::Port_Scan']
zeek_address_scan = zeek_address_scan.drop_duplicates(subset='src', keep='frist')
zeek_address_scan = zeek_address_scan.sort_values(by='ts', ascending=True)
zeek_address_scan = zeek_address_scan.reset_index(drop=True)

conn: pd.DataFrame = load_latest_dataframe(LogNamesEnum.conn)
conn_udp: pd.DataFrame = conn[conn['proto'] == 'udp']
conn_udp_icmp: pd.DataFrame = conn[conn['proto'].isin(['udp', 'icmp'])]

In [ ]:
from src.d04_analysation.find_scanner import *
from src.d01_data.log_names_enum import *
from src.d01_data.save_files import *
from collections import defaultdict
from typing import Tuple

address_scanner_dict: defaultdict[Tuple[str, int], pd.DataFrame] = defaultdict(pd.DataFrame)
port_scanner_dict: defaultdict[Tuple[str, int], pd.DataFrame] = defaultdict(pd.DataFrame)

scanner: pd.DataFrame = pd.DataFrame({
        'threshold': pd.Series(dtype='int'),
        'interval': pd.Series(dtype='int'),
        'address_scanner': pd.Series(dtype='object'),
        'port_scanner': pd.Series(dtype='object'),
    })
for threshold in range(2, 50, 2):
    for interval in range(60 - 1860, 60):
        print(f'working on threshold: {threshold} and interval: {interval}')
        find_scanner_return: FindScannerReturn = find_scanning(unique_port_scanner=zeek_port_scan, unique_address_scanner=zeek_address_scan, conn_udp=conn_udp, address_threshold=threshold, port_threshold=threshold, address_interval=interval, port_interval=interval)
        new_row: pd.DataFrame = pd.DataFrame({
            'threshold': [threshold],
            'address_scanner': [find_scanner_return.address_scanner],
            'port_scanner': [find_scanner_return.port_scanner],
        })
        scanner = pd.concat([scanner, new_row], ignore_index=True)
        if threshold % 5 == 0:
            print(f'saving threshold: {threshold} and interval: {interval}')
            save_dataframe_as_feather(scanner, LogNamesEnum.scanner, f't{threshold}i{interval}')
save_dataframe_as_feather(scanner, LogNamesEnum.scanner, f'final')